In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
import pandas as pd


In [ ]:
from openpyxl import load_workbook
import pandas as pd

file_path = r"C:\Users\91700\Desktop\AEGIS\oct2024_feb2025.xlsx"

try:
    # Try loading with openpyxl first
    wb = load_workbook(file_path)
    print(f"Successfully loaded workbook '{file_path}' with openpyxl.")
    # If successful, you can then proceed to use pd.read_excel or openpyxl directly
    # For pd.read_excel, openpyxl is often the default engine for .xlsx
    df = pd.read_excel(file_path, engine='openpyxl')
    print("DataFrame loaded successfully with pandas.")
    print(df.head())

except Exception as e:
    print(f"An error occurred: {e}")
    # If openpyxl fails, try pandas with a different engine if applicable
    # This might help if there's a specific issue with openpyxl's parser
    # for certain malformed files, but unlikely for standard .xlsx
    try:
        print("\nAttempting to load with pandas default engine (might use xlrd for older formats, or openpyxl):")
        df_pd_default = pd.read_excel(file_path)
        print("DataFrame loaded successfully with pandas default engine.")
        print(df_pd_default.head())
    except Exception as e_pd:
        print(f"Pandas default engine also failed: {e_pd}")
        

In [ ]:
df.info()

In [ ]:
print(df.columns.tolist())

In [ ]:
import pandas as pd

# --- Step 1: Convert relevant columns to numeric type ---
# This is crucial as financial data might load as strings due to commas, currency symbols, or mixed types.
# .astype(str) ensures all values are treated as strings before replacement.
# .str.replace('[^0-9.-]', '', regex=True) removes any character that is NOT a digit, a period (.), or a hyphen (-).
# errors='coerce' will convert any values that cannot be parsed into a number (after cleaning) into NaN (Not a Number).
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
df['NET_DEALER_PAYOUT'] = pd.to_numeric(df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

# --- Step 2: Filter the DataFrame based on the condition ---
# Select rows where 'INS_PAYOUT_IN_Rs' is strictly less than 'NET_DEALER_PAYOUT'.
# Also, ensure both values are not NaN (missing) to avoid unexpected comparison behavior.
filtered_df = df[
    (df['INS_PAYOUT_IN_Rs'] < df['NET_DEALER_PAYOUT']) &
    (df['INS_PAYOUT_IN_Rs'].notna()) &
    (df['NET_DEALER_PAYOUT'].notna())
]

# --- Step 3: Select the desired columns from the filtered DataFrame ---
# From the rows that meet the condition, extract only the 'Master_Dealer_Name' and 'Insurance_Policy_No'.
result_df = filtered_df[['Master_Dealer_Name', 'Insurance_Policy_No']]

# --- Step 4: (Optional but Recommended) Get unique combinations ---
# It's common for a Master Dealer and Policy No pair to appear multiple times if there are
# multiple entries for the same policy/dealer. drop_duplicates() ensures each unique pair is listed only once.
unique_combinations = result_df.drop_duplicates().reset_index(drop=True)

# --- Step 5: Display the results ---
print("Master Dealer Name and Insurance Policy No. where INS_PAYOUT_IN_Rs < NET_DEALER_PAYOUT:")
print(unique_combinations)

# Optional: You can also save this result to a new Excel file
# unique_combinations.to_excel("Filtered_Master_Dealer_Policies.xlsx", index=False)
# print("\nResult saved to Filtered_Master_Dealer_Policies.xlsx")

In [ ]:
import pandas as pd

# --- 1. Data Type Conversion (Crucial for numerical comparison) ---
# Convert 'INS_PAYOUT_IN_Rs' and 'NET_DEALER_PAYOUT' to numeric.
# This handles potential non-numeric characters and converts unparseable values to NaN.
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
df['NET_DEALER_PAYOUT'] = pd.to_numeric(df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

# --- 2. Calculate Total Unique Policies Issued by each Master Dealer ---
# Group by 'Master_Dealer_Name' and count unique 'Insurance_Policy_No'.
total_policies_issued = df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
total_policies_issued.rename(columns={'Insurance_Policy_No': 'NO OF POLICY ISSUED IN TOTAL'}, inplace=True)

# --- 3. Filter Data based on the Condition (INS_PAYOUT_IN_Rs < NET_DEALER_PAYOUT) ---
# Select rows where INS_PAYOUT_IN_Rs is less than NET_DEALER_PAYOUT and neither is NaN.
condition_satisfied_df = df[
    (df['INS_PAYOUT_IN_Rs'] < df['NET_DEALER_PAYOUT']) &
    (df['INS_PAYOUT_IN_Rs'].notna()) &
    (df['NET_DEALER_PAYOUT'].notna())
]

# --- 4. Count Unique Policies that Satisfied the Condition for each Master Dealer ---
# Group the filtered data and count unique 'Insurance_Policy_No'.
policies_satisfying_condition = condition_satisfied_df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
policies_satisfying_condition.rename(columns={'Insurance_Policy_No': 'CONDITION SATISFIED'}, inplace=True)

# --- 5. Merge the two counts and determine "PROBLEM" status ---
# Merge the total policies and policies satisfying the condition based on 'Master_Dealer_Name'.
# Use a 'left' merge to ensure all master dealers from the total list are included.
final_report = pd.merge(
    total_policies_issued,
    policies_satisfying_condition,
    on='Master_Dealer_Name',
    how='left'
)

# Fill NaN values in 'CONDITION SATISFIED' with 0 (for dealers who had no policies satisfying the condition)
final_report['CONDITION SATISFIED'] = final_report['CONDITION SATISFIED'].fillna(0).astype(int)

# Determine the 'PROBLEM' status
# If 'NO OF POLICY ISSUED IN TOTAL' == 'CONDITION SATISFIED', then 'SOLVED', else 'FAILED'.
final_report['PROBLEM'] = final_report.apply(
    lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
    axis=1
)

# --- 6. Display the final result ---
print("Master Dealer Policy Analysis:")
print(final_report)

# Optional: You can also save this report to an Excel file
# final_report.to_excel("Master_Dealer_Policy_Analysis.xlsx", index=False)
# print("\nReport saved to Master_Dealer_Policy_Analysis.xlsx")

In [ ]:
import pandas as pd

# --- 1. Data Type Conversion (Crucial for numerical comparison) ---
# Convert 'INS_PAYOUT_IN_Rs' and 'NET_DEALER_PAYOUT' columns to numeric types.
# This handles cases where values might be strings due to formatting (e.g., commas, currency symbols).
# .astype(str) ensures all values are treated as strings before replacement.
# .str.replace('[^0-9.-]', '', regex=True) removes any character that is NOT a digit, a period (.), or a hyphen (-).
# errors='coerce' will turn any values that cannot be parsed into a number (after cleaning) into NaN.
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
df['NET_DEALER_PAYOUT'] = pd.to_numeric(df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

# --- 2. Calculate Total Unique Policies Issued by each Master Dealer ---
# Group the DataFrame by 'Master_Dealer_Name' and count the number of unique 'Insurance_Policy_No' for each dealer.
total_policies_issued = df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
# Rename the column to match the desired output format.
total_policies_issued.rename(columns={'Insurance_Policy_No': 'NO OF POLICY ISSUED IN TOTAL'}, inplace=True)

# --- 3. Filter Data based on the Condition (INS_PAYOUT_IN_Rs < NET_DEALER_PAYOUT) ---
# Create a filtered DataFrame containing only the rows where the specified condition is met.
# Also, ensure that both payout values are not NaN (missing) to avoid errors during comparison.
condition_satisfied_df = df[
    (df['INS_PAYOUT_IN_Rs'] < df['NET_DEALER_PAYOUT']) &
    (df['INS_PAYOUT_IN_Rs'].notna()) &
    (df['NET_DEALER_PAYOUT'].notna())
]

# --- 4. Count Unique Policies that Satisfied the Condition for each Master Dealer ---
# Group the *filtered* DataFrame by 'Master_Dealer_Name' and count unique 'Insurance_Policy_No' that met the condition.
policies_satisfying_condition = condition_satisfied_df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
# Rename the column to match the desired output format.
policies_satisfying_condition.rename(columns={'Insurance_Policy_No': 'CONDITION SATISFIED'}, inplace=True)

# --- 5. Merge the two counts and determine "PROBLEM" status ---
# Perform a left merge to combine the 'total_policies_issued' with 'policies_satisfying_condition'.
# A left merge ensures all master dealers from the total list are included, even if they have no policies satisfying the condition.
final_report = pd.merge(
    total_policies_issued,
    policies_satisfying_condition,
    on='Master_Dealer_Name',
    how='left'
)

# Fill any NaN values in 'CONDITION SATISFIED' (which occur if a dealer had no policies meeting the condition) with 0.
# Then convert the column to an integer type for cleaner display.
final_report['CONDITION SATISFIED'] = final_report['CONDITION SATISFIED'].fillna(0).astype(int)

# Create the 'PROBLEM' column based on the comparison of the two counts.
# If the total policies issued matches the policies satisfying the condition, it's 'SOLVED', otherwise 'FAILED'.
final_report['PROBLEM'] = final_report.apply(
    lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
    axis=1
)

# --- 6. Display the final report in a proper table format ---
print("--- Master Dealer Policy Analysis ---")
print(final_report.to_string(index=False)) # .to_string(index=False) for clean, non-indexed output

# --- 7. Feature to Export the Report ---
output_file_name = "Master_Dealer_Policy_Analysis_Report.xlsx"
try:
    final_report.to_excel(output_file_name, index=False)
    print(f"\nReport successfully exported to '{output_file_name}'")
    print(f"You can find this file in the same directory where your Python script is running.")
except Exception as e:
    print(f"\nError exporting report to Excel: {e}")
    print("Please ensure the file is not open and you have write permissions to the directory.")

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import IntText, Dropdown, Output, VBox, HBox
from IPython.display import display, clear_output

# --- Perform the data processing ---
# 1. Data Type Conversion for Payouts
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
df['NET_DEALER_PAYOUT'] = pd.to_numeric(df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

# --- Convert 'Month' (DD-MM-YYYY) column to 'Month_Year' format ---
date_column_name_in_df = 'Month'

if date_column_name_in_df in df.columns:
    df[date_column_name_in_df] = pd.to_datetime(df[date_column_name_in_df], format='%d-%m-%Y', errors='coerce')
    df['Month_Year'] = df[date_column_name_in_df].dt.strftime('%b %Y')
    df.dropna(subset=[date_column_name_in_df], inplace=True)
else:
    raise ValueError(f"Error: Date column '{date_column_name_in_df}' not found in your DataFrame. Please ensure this column exists and contains dates in 'DD-MM-YYYY' format.")

# ----------------------------------------------------------------------------------

# --- Create ipywidgets for filters ---
unique_month_periods = df[date_column_name_in_df].dt.to_period('M').unique().tolist()
unique_month_periods.sort()
month_year_options = [str(m.strftime('%b %Y')) for m in unique_month_periods]

month_year_dropdown = Dropdown(
    options=['All'] + month_year_options,
    value='All',
    description='Month & Year:'
)

problem_dropdown = Dropdown(
    options=['All', 'SOLVED', 'FAILED'],
    value='All',
    description='Problem Status:'
)

min_policy_count_input = IntText(
    value=0,
    description='Min Policies:',
    continuous_update=False
)

max_policy_count_input = IntText(
    value=int(df['Insurance_Policy_No'].nunique()),
    description='Max Policies:',
    continuous_update=False
)

plot_output = Output()

# --- Function to update the plot based on filter selections ---
def update_plot(*args):
    with plot_output:
        clear_output(wait=True)

        selected_month_year = month_year_dropdown.value
        selected_problem = problem_dropdown.value
        min_count = min_policy_count_input.value
        max_count = max_policy_count_input.value

        filtered_df_base = df.copy()

        if selected_month_year != 'All':
            filtered_df_base = filtered_df_base[filtered_df_base['Month_Year'] == selected_month_year]

        if filtered_df_base.empty:
            print(f"No data for selected Month & Year: {selected_month_year}. Please adjust filters.")
            return

        temp_total_policies_issued = filtered_df_base.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
        temp_total_policies_issued.rename(columns={'Insurance_Policy_No': 'NO OF POLICY ISSUED IN TOTAL'}, inplace=True)

        temp_condition_satisfied_df = filtered_df_base[
            (filtered_df_base['INS_PAYOUT_IN_Rs'] < filtered_df_base['NET_DEALER_PAYOUT']) &
            (filtered_df_base['INS_PAYOUT_IN_Rs'].notna()) &
            (filtered_df_base['NET_DEALER_PAYOUT'].notna())
        ]
        temp_policies_satisfying_condition = temp_condition_satisfied_df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
        temp_policies_satisfying_condition.rename(columns={'Insurance_Policy_No': 'CONDITION SATISFIED'}, inplace=True)

        temp_final_report = pd.merge(
            temp_total_policies_issued,
            temp_policies_satisfying_condition,
            on='Master_Dealer_Name',
            how='left'
        )

        # --- ROBUST CONVERSION FOR 'CONDITION SATISFIED' ---
        # Ensure the column is numeric (float) first, then fill NaNs, then convert to int.
        temp_final_report['CONDITION SATISFIED'] = pd.to_numeric(temp_final_report['CONDITION SATISFIED'], errors='coerce')
        temp_final_report['CONDITION SATISFIED'] = temp_final_report['CONDITION SATISFIED'].fillna(0).astype(int)
        # --- END ROBUST CONVERSION ---

        temp_final_report['PROBLEM'] = temp_final_report.apply(
            lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
            axis=1
        )

        if selected_problem != 'All':
            temp_final_report = temp_final_report[temp_final_report['PROBLEM'] == selected_problem]

        temp_final_report = temp_final_report[
            (temp_final_report['NO OF POLICY ISSUED IN TOTAL'] >= min_count) &
            (temp_final_report['NO OF POLICY ISSUED IN TOTAL'] <= max_count)
        ]

        if temp_final_report.empty:
            print("No data to display for the selected combination of filters.")
            return

        melted_df_filtered = temp_final_report.melt(
            id_vars=['Master_Dealer_Name', 'PROBLEM'],
            value_vars=['NO OF POLICY ISSUED IN TOTAL', 'CONDITION SATISFIED'],
            var_name='Metric',
            value_name='Count'
        )

        melted_df_filtered = melted_df_filtered.sort_values(by=['Master_Dealer_Name', 'Metric'])

        fig = px.bar(
            melted_df_filtered,
            x='Master_Dealer_Name',
            y='Count',
            color='Metric',
            barmode='group',
            title=f'Policy Comparison: Total vs. Condition-Satisfied<br><sup>Month/Year={selected_month_year}, Problem={selected_problem}, Policies={min_count}-{max_count}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Count': 'Number of Unique Policies',
                'Metric': 'Policy Type'
            },
            hover_data={
                'Master_Dealer_Name': True,
                'Metric': True,
                'Count': True,
                'PROBLEM': True
            },
            category_orders={
                "Metric": ['NO OF POLICY ISSUED IN TOTAL', 'CONDITION SATISFIED']
            },
            color_discrete_map={
                'NO OF POLICY ISSUED IN TOTAL': 'steelblue',
                'CONDITION SATISFIED': 'darkorange'
            }
        )

        fig.update_layout(
            xaxis_title_text='Master Dealer Name',
            yaxis_title_text='Number of Unique Policies',
            font=dict(size=10),
            title_x=0.5,
            height=1100,
            width=1800,
            hovermode="x unified",
            legend_title_text="Policy Type"
        )

        fig.update_xaxes(tickangle=-45)
        fig.update_traces(texttemplate='%{y}', textposition='outside')

        fig.show()

month_year_dropdown.observe(update_plot, names='value')
problem_dropdown.observe(update_plot, names='value')
min_policy_count_input.observe(update_plot, names='value')
max_policy_count_input.observe(update_plot, names='value')

display(HBox([month_year_dropdown, problem_dropdown, min_policy_count_input, max_policy_count_input]))
display(plot_output)

update_plot()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import IntText, Dropdown, Output, VBox, HBox
from IPython.display import display, clear_output


# --- Perform the data processing ---
# 1. Data Type Conversion for Payouts
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
df['NET_DEALER_PAYOUT'] = pd.to_numeric(df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

# --- Convert 'Month' (DD-MM-YYYY) column to 'Month_Year' format ---
date_column_name_in_df = 'Month'

if date_column_name_in_df in df.columns:
    df[date_column_name_in_df] = pd.to_datetime(df[date_column_name_in_df], format='%d-%m-%Y', errors='coerce')
    df['Month_Year'] = df[date_column_name_in_df].dt.strftime('%b %Y')
    df.dropna(subset=[date_column_name_in_df], inplace=True)
else:
    raise ValueError(f"Error: Date column '{date_column_name_in_df}' not found in your DataFrame. Please ensure this column exists and contains dates in 'DD-MM-YYYY' format.")

# ----------------------------------------------------------------------------------

# --- Create ipywidgets for filters ---
unique_month_periods = df[date_column_name_in_df].dt.to_period('M').unique().tolist()
unique_month_periods.sort()
month_year_options = [str(m.strftime('%b %Y')) for m in unique_month_periods]

month_year_dropdown = Dropdown(
    options=['All'] + month_year_options,
    value='All',
    description='Month & Year:'
)

problem_dropdown = Dropdown(
    options=['All', 'SOLVED', 'FAILED'],
    value='All',
    description='Problem Status:'
)

min_policy_count_input = IntText(
    value=0,
    description='Min Policies:',
    continuous_update=False
)

max_policy_count_input = IntText(
    value=int(df['Insurance_Policy_No'].nunique()),
    description='Max Policies:',
    continuous_update=False
)

plot_output_policies = Output() # Output for the first plot (Policy Counts)
plot_output_payouts = Output()  # New Output for the second plot (Payout Sums)

# --- Function to update both plots based on filter selections ---
def update_plot(*args):
    # Clear both output areas
    with plot_output_policies:
        clear_output(wait=True)
    with plot_output_payouts:
        clear_output(wait=True)

    # Get filter values
    selected_month_year = month_year_dropdown.value
    selected_problem = problem_dropdown.value
    min_count = min_policy_count_input.value
    max_count = max_policy_count_input.value

    # Start with a copy of the original DataFrame for filtering
    filtered_df_base = df.copy()

    # Apply Month & Year filter
    if selected_month_year != 'All':
        filtered_df_base = filtered_df_base[filtered_df_base['Month_Year'] == selected_month_year]

    if filtered_df_base.empty:
        with plot_output_policies:
            print(f"No data for selected Month & Year: {selected_month_year}. Please adjust filters.")
        with plot_output_payouts:
            print(f"No data for selected Month & Year: {selected_month_year}. Please adjust filters.")
        return

    # --- Data Processing for Policy Counts (First Plot) ---
    temp_total_policies_issued = filtered_df_base.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
    temp_total_policies_issued.rename(columns={'Insurance_Policy_No': 'NO OF POLICY ISSUED IN TOTAL'}, inplace=True)

    temp_condition_satisfied_df = filtered_df_base[
        (filtered_df_base['INS_PAYOUT_IN_Rs'] < filtered_df_base['NET_DEALER_PAYOUT']) &
        (filtered_df_base['INS_PAYOUT_IN_Rs'].notna()) &
        (filtered_df_base['NET_DEALER_PAYOUT'].notna())
    ]
    temp_policies_satisfying_condition = temp_condition_satisfied_df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
    temp_policies_satisfying_condition.rename(columns={'Insurance_Policy_No': 'CONDITION SATISFIED'}, inplace=True)

    temp_final_report = pd.merge(
        temp_total_policies_issued,
        temp_policies_satisfying_condition,
        on='Master_Dealer_Name',
        how='left'
    )

    temp_final_report['CONDITION SATISFIED'] = pd.to_numeric(temp_final_report['CONDITION SATISFIED'], errors='coerce')
    temp_final_report['CONDITION SATISFIED'] = temp_final_report['CONDITION SATISFIED'].fillna(0).astype(int)

    temp_final_report['PROBLEM'] = temp_final_report.apply(
        lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
        axis=1
    )

    # Apply Problem Status filter to temp_final_report
    if selected_problem != 'All':
        temp_final_report = temp_final_report[temp_final_report['PROBLEM'] == selected_problem]

    # Apply Policy Count filter to temp_final_report
    temp_final_report = temp_final_report[
        (temp_final_report['NO OF POLICY ISSUED IN TOTAL'] >= min_count) &
        (temp_final_report['NO OF POLICY ISSUED IN TOTAL'] <= max_count)
    ]

    # --- Data Processing for Payout Sums (Second Plot) ---
    payouts_summed = filtered_df_base.groupby('Master_Dealer_Name')[
        ['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT']
    ].sum().reset_index()

    payouts_for_plot = pd.merge(
        temp_final_report[['Master_Dealer_Name', 'PROBLEM']],
        payouts_summed,
        on='Master_Dealer_Name',
        how='inner'
    )

    if temp_final_report.empty:
        with plot_output_policies:
            print("No data to display for the selected combination of filters for Policy Counts.")
        with plot_output_payouts:
            print("No data to display for the selected combination of filters for Payout Sums.")
        return

    melted_df_policies = temp_final_report.melt(
        id_vars=['Master_Dealer_Name', 'PROBLEM'],
        value_vars=['NO OF POLICY ISSUED IN TOTAL', 'CONDITION SATISFIED'],
        var_name='Metric',
        value_name='Count'
    )
    melted_df_policies = melted_df_policies.sort_values(by=['Master_Dealer_Name', 'Metric'])


    if payouts_for_plot.empty:
        with plot_output_policies:
            pass
        with plot_output_payouts:
            print("No data to display for the selected combination of filters for Payout Sums.")
        return

    melted_df_payouts = payouts_for_plot.melt(
        id_vars=['Master_Dealer_Name', 'PROBLEM'],
        value_vars=['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT'],
        var_name='Metric',
        value_name='Sum'
    )
    melted_df_payouts = melted_df_payouts.sort_values(by=['Master_Dealer_Name', 'Metric'])


    # --- Plotting the first chart (Policy Counts) ---
    with plot_output_policies:
        fig_policies = px.bar(
            melted_df_policies,
            x='Master_Dealer_Name',
            y='Count',
            color='Metric',
            barmode='group',
            title=f'Policy Comparison: Total vs. Condition-Satisfied<br><sup>Month/Year={selected_month_year}, Problem={selected_problem}, Policies={min_count}-{max_count}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Count': 'Number of Unique Policies',
                'Metric': 'Policy Type'
            },
            hover_data={
                'Master_Dealer_Name': True, 'Metric': True, 'Count': True, 'PROBLEM': True
            },
            category_orders={"Metric": ['NO OF POLICY ISSUED IN TOTAL', 'CONDITION SATISFIED']},
            color_discrete_map={
                'NO OF POLICY ISSUED IN TOTAL': 'steelblue',
                'CONDITION SATISFIED': 'darkorange'
            }
        )
        fig_policies.update_layout(
            xaxis_title_text='Master Dealer Name', yaxis_title_text='Number of Unique Policies',
            font=dict(size=10),
            height=1000, # ENSURING CONSISTENT HEIGHT
            width=1800,  # ENSURING CONSISTENT WIDTH
            title_x=0.5, hovermode="x unified", legend_title_text="Policy Type"
        )
        fig_policies.update_xaxes(tickangle=-45)
        fig_policies.update_traces(texttemplate='%{y}', textposition='outside')
        fig_policies.show()

    # --- Plotting the second chart (Payout Sums) ---
    with plot_output_payouts:
        fig_payouts = px.bar(
            melted_df_payouts,
            x='Master_Dealer_Name',
            y='Sum',
            color='Metric',
            barmode='group',
            title=f'Payout Comparison: INS_PAYOUT_IN_Rs vs. NET_DEALER_PAYOUT<br><sup>Month/Year={selected_month_year}, Problem={selected_problem}, Policies={min_count}-{max_count}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Sum': 'Total Amount (INR)',
                'Metric': 'Payout Type'
            },
            hover_data={
                'Master_Dealer_Name': True, 'Metric': True, 'Sum': True, 'PROBLEM': True
            },
            category_orders={
                "Metric": ['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT']
            },
            color_discrete_map={
                'INS_PAYOUT_IN_Rs': 'cyan',
                'NET_DEALER_PAYOUT': 'red'
            }
        )
        fig_payouts.update_layout(
            xaxis_title_text='Master Dealer Name', yaxis_title_text='Total Amount (INR)',
            font=dict(size=10),
            height=1000, # ENSURING CONSISTENT HEIGHT
            width=1800,  # ENSURING CONSISTENT WIDTH
            title_x=0.8, hovermode="x unified", legend_title_text="Payout Type"
        )
        fig_payouts.update_xaxes(tickangle=-45)
        fig_payouts.update_traces(texttemplate='%{y:.2s}', textposition='outside')
        fig_payouts.show()


# --- Attach update triggers to widgets ---
month_year_dropdown.observe(update_plot, names='value')
problem_dropdown.observe(update_plot, names='value')
min_policy_count_input.observe(update_plot, names='value')
max_policy_count_input.observe(update_plot, names='value')

# --- Initial display of widgets and plots ---
display(HBox([month_year_dropdown, problem_dropdown, min_policy_count_input, max_policy_count_input]))
display(VBox([plot_output_policies, plot_output_payouts]))

# Trigger initial plot display
update_plot()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import IntText, Dropdown, Output, VBox, HBox
from IPython.display import display, clear_output

# --- Perform the data processing ---
# 1. Data Type Conversion for Payouts and NET_REVENUE
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
df['NET_DEALER_PAYOUT'] = pd.to_numeric(df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

# Add NET_REVENUE conversion here. If NET_REVENUE is a calculated field,
# perform the calculation here as well. For example:
# df['NET_REVENUE'] = df['NET_DEALER_PAYOUT'] - df['SOME_COST_COLUMN']
if 'NET_REVENUE' in df.columns:
    df['NET_REVENUE'] = pd.to_numeric(df['NET_REVENUE'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
else:
    # If NET_REVENUE column is missing, create a dummy one for demonstration
    # or handle it according to your data structure.
    print("Warning: 'NET_REVENUE' column not found. Creating dummy data for demonstration.")
    df['NET_REVENUE'] = df['NET_DEALER_PAYOUT'] * 0.8 # Example dummy calculation


# --- Convert 'Month' (DD-MM-YYYY) column to 'Month_Year' format ---
date_column_name_in_df = 'Month'

if date_column_name_in_df in df.columns:
    df[date_column_name_in_df] = pd.to_datetime(df[date_column_name_in_df], format='%d-%m-%Y', errors='coerce')
    df['Month_Year'] = df[date_column_name_in_df].dt.strftime('%b %Y')
    df.dropna(subset=[date_column_name_in_df], inplace=True)
else:
    raise ValueError(f"Error: Date column '{date_column_name_in_df}' not found in your DataFrame. Please ensure this column exists and contains dates in 'DD-MM-YYYY' format.")

# ----------------------------------------------------------------------------------

# --- Create ipywidgets for filters ---
unique_month_periods = df[date_column_name_in_df].dt.to_period('M').unique().tolist()
unique_month_periods.sort()
month_year_options = [str(m.strftime('%b %Y')) for m in unique_month_periods]

month_year_dropdown = Dropdown(
    options=['All'] + month_year_options,
    value='All',
    description='Month & Year:'
)

problem_dropdown = Dropdown(
    options=['All', 'SOLVED', 'FAILED'],
    value='All',
    description='Problem Status:'
)

min_policy_count_input = IntText(
    value=0,
    description='Min Policies:',
    continuous_update=False
)

max_policy_count_input = IntText(
    value=int(df['Insurance_Policy_No'].nunique()),
    description='Max Policies:',
    continuous_update=False
)

plot_output_policies = Output() # Output for the first plot (Policy Counts)
plot_output_payouts = Output()  # Output for the second plot (Payout Sums)
plot_output_net_revenue = Output() # NEW Output for the third plot (Net Revenue)

# --- Function to update all plots based on filter selections ---
def update_plot(*args):
    # Clear all output areas
    with plot_output_policies:
        clear_output(wait=True)
    with plot_output_payouts:
        clear_output(wait=True)
    with plot_output_net_revenue: # Clear new plot output
        clear_output(wait=True)

    # Get filter values
    selected_month_year = month_year_dropdown.value
    selected_problem = problem_dropdown.value
    min_count = min_policy_count_input.value
    max_count = max_policy_count_input.value

    # Start with a copy of the original DataFrame for filtering
    filtered_df_base = df.copy()

    # Apply Month & Year filter
    if selected_month_year != 'All':
        filtered_df_base = filtered_df_base[filtered_df_base['Month_Year'] == selected_month_year]

    if filtered_df_base.empty:
        with plot_output_policies:
            print(f"No data for selected Month & Year: {selected_month_year}. Please adjust filters.")
        with plot_output_payouts:
            print(f"No data for selected Month & Year: {selected_month_year}. Please adjust filters.")
        with plot_output_net_revenue:
            print(f"No data for selected Month & Year: {selected_month_year}. Please adjust filters.")
        return

    # --- Data Processing for Policy Counts (First Plot) ---
    temp_total_policies_issued = filtered_df_base.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
    temp_total_policies_issued.rename(columns={'Insurance_Policy_No': 'NO OF POLICY ISSUED IN TOTAL'}, inplace=True)

    temp_condition_satisfied_df = filtered_df_base[
        (filtered_df_base['INS_PAYOUT_IN_Rs'] < filtered_df_base['NET_DEALER_PAYOUT']) &
        (filtered_df_base['INS_PAYOUT_IN_Rs'].notna()) &
        (filtered_df_base['NET_DEALER_PAYOUT'].notna())
    ]
    temp_policies_satisfying_condition = temp_condition_satisfied_df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
    temp_policies_satisfying_condition.rename(columns={'Insurance_Policy_No': 'CONDITION SATISFIED'}, inplace=True)

    temp_final_report = pd.merge(
        temp_total_policies_issued,
        temp_policies_satisfying_condition,
        on='Master_Dealer_Name',
        how='left'
    )

    temp_final_report['CONDITION SATISFIED'] = pd.to_numeric(temp_final_report['CONDITION SATISFIED'], errors='coerce')
    temp_final_report['CONDITION SATISFIED'] = temp_final_report['CONDITION SATISFIED'].fillna(0).astype(int)

    temp_final_report['PROBLEM'] = temp_final_report.apply(
        lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
        axis=1
    )

    # Apply Problem Status filter to temp_final_report
    if selected_problem != 'All':
        temp_final_report = temp_final_report[temp_final_report['PROBLEM'] == selected_problem]

    # Apply Policy Count filter to temp_final_report
    temp_final_report = temp_final_report[
        (temp_final_report['NO OF POLICY ISSUED IN TOTAL'] >= min_count) &
        (temp_final_report['NO OF POLICY ISSUED IN TOTAL'] <= max_count)
    ]

    # --- Data Processing for Payout Sums (Second Plot) ---
    payouts_summed = filtered_df_base.groupby('Master_Dealer_Name')[
        ['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT']
    ].sum().reset_index()

    payouts_for_plot = pd.merge(
        temp_final_report[['Master_Dealer_Name', 'PROBLEM']],
        payouts_summed,
        on='Master_Dealer_Name',
        how='inner'
    )

    # --- Data Processing for NET_REVENUE Sum (Third Plot) ---
    net_revenue_summed = filtered_df_base.groupby('Master_Dealer_Name')[
        'NET_REVENUE'
    ].sum().reset_index()

    net_revenue_for_plot = pd.merge(
        temp_final_report[['Master_Dealer_Name', 'PROBLEM']],
        net_revenue_summed,
        on='Master_Dealer_Name',
        how='inner'
    )


    # Check if data exists after all filtering before plotting
    if temp_final_report.empty:
        with plot_output_policies:
            print("No data to display for the selected combination of filters for Policy Counts.")
        with plot_output_payouts:
            print("No data to display for the selected combination of filters for Payout Sums.")
        with plot_output_net_revenue:
            print("No data to display for the selected combination of filters for Net Revenue Sum.")
        return

    # Melt the filtered data for policy counts plot
    melted_df_policies = temp_final_report.melt(
        id_vars=['Master_Dealer_Name', 'PROBLEM'],
        value_vars=['NO OF POLICY ISSUED IN TOTAL', 'CONDITION SATISFIED'],
        var_name='Metric',
        value_name='Count'
    )
    melted_df_policies = melted_df_policies.sort_values(by=['Master_Dealer_Name', 'Metric'])


    # Melt the filtered data for payout sums plot
    if payouts_for_plot.empty:
        with plot_output_payouts:
            print("No data to display for the selected combination of filters for Payout Sums.")
    melted_df_payouts = payouts_for_plot.melt(
        id_vars=['Master_Dealer_Name', 'PROBLEM'],
        value_vars=['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT'],
        var_name='Metric',
        value_name='Sum'
    )
    melted_df_payouts = melted_df_payouts.sort_values(by=['Master_Dealer_Name', 'Metric'])

    # Melt the filtered data for net revenue plot
    if net_revenue_for_plot.empty:
        with plot_output_net_revenue:
            print("No data to display for the selected combination of filters for Net Revenue Sum.")
    # For a single metric, melting is not strictly necessary but can be used for consistency with previous plots
    melted_df_net_revenue = net_revenue_for_plot.melt(
        id_vars=['Master_Dealer_Name', 'PROBLEM'],
        value_vars=['NET_REVENUE'],
        var_name='Metric', # This will just be 'NET_REVENUE'
        value_name='Sum'
    )
    melted_df_net_revenue = melted_df_net_revenue.sort_values(by=['Master_Dealer_Name'])


    # --- Plotting the first chart (Policy Counts) ---
    with plot_output_policies:
        fig_policies = px.bar(
            melted_df_policies,
            x='Master_Dealer_Name',
            y='Count',
            color='Metric',
            barmode='group',
            title=f'Policy Comparison: Total vs. Condition-Satisfied<br><sup>Month/Year={selected_month_year}, Problem={selected_problem}, Policies={min_count}-{max_count}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Count': 'Number of Unique Policies',
                'Metric': 'Policy Type'
            },
            hover_data={
                'Master_Dealer_Name': True, 'Metric': True, 'Count': True, 'PROBLEM': True
            },
            category_orders={"Metric": ['NO OF POLICY ISSUED IN TOTAL', 'CONDITION SATISFIED']},
            color_discrete_map={
                'NO OF POLICY ISSUED IN TOTAL': 'steelblue',
                'CONDITION SATISFIED': 'darkorange'
            }
        )
        fig_policies.update_layout(
            xaxis_title_text='Master Dealer Name', yaxis_title_text='Number of Unique Policies',
            font=dict(size=10),
            height=1000,
            width=1900,
            title_x=0.5, hovermode="x unified", legend_title_text="Policy Type"
        )
        fig_policies.update_xaxes(tickangle=-45)
        fig_policies.update_traces(texttemplate='%{y}', textposition='outside')
        fig_policies.show()

    # --- Plotting the second chart (Payout Sums) ---
    with plot_output_payouts:
        fig_payouts = px.bar(
            melted_df_payouts,
            x='Master_Dealer_Name',
            y='Sum',
            color='Metric',
            barmode='group',
            title=f'Payout Comparison: INS_PAYOUT_IN_Rs vs. NET_DEALER_PAYOUT<br><sup>Month/Year={selected_month_year}, Problem={selected_problem}, Policies={min_count}-{max_count}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Sum': 'Total Amount (INR)',
                'Metric': 'Payout Type'
            },
            hover_data={
                'Master_Dealer_Name': True, 'Metric': True, 'Sum': True, 'PROBLEM': True
            },
            category_orders={
                "Metric": ['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT']
            },
            color_discrete_map={
                'INS_PAYOUT_IN_Rs': 'cyan',
                'NET_DEALER_PAYOUT': 'darkred'
            }
        )
        fig_payouts.update_layout(
            xaxis_title_text='Master Dealer Name', yaxis_title_text='Total Amount (INR)',
            font=dict(size=10),
            height=1000,
            width=1900,
            title_x=0.5, hovermode="x unified", legend_title_text="Payout Type"
        )
        fig_payouts.update_xaxes(tickangle=-45)
        fig_payouts.update_traces(texttemplate='%{y:.2s}', textposition='outside')
        fig_payouts.show()

    # --- Plotting the third chart (NET_REVENUE Sum) ---
    with plot_output_net_revenue:
        fig_net_revenue = px.bar(
            melted_df_net_revenue,
            x='Master_Dealer_Name',
            y='Sum',
            color_discrete_sequence=['purple'], # Single color for single metric
            title=f'Total NET_REVENUE by Master Dealer<br><sup>Month/Year={selected_month_year}, Problem={selected_problem}, Policies={min_count}-{max_count}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Sum': 'Total NET_REVENUE (INR)',
                'Metric': 'Revenue Type' # Though it's just one type
            },
            hover_data={
                'Master_Dealer_Name': True, 'Sum': True, 'PROBLEM': True
            }
        )
        fig_net_revenue.update_layout(
            xaxis_title_text='Master Dealer Name', yaxis_title_text='Total NET_REVENUE (INR)',
            font=dict(size=10),
            height=1000,
            width=1900,
            title_x=0.5, hovermode="x unified"
        )
        fig_net_revenue.update_xaxes(tickangle=-45)
        fig_net_revenue.update_traces(texttemplate='%{y:.2s}', textposition='outside')
        fig_net_revenue.show()


# --- Attach update triggers to widgets ---
month_year_dropdown.observe(update_plot, names='value')
problem_dropdown.observe(update_plot, names='value')
min_policy_count_input.observe(update_plot, names='value')
max_policy_count_input.observe(update_plot, names='value')

# --- Initial display of widgets and plots ---
display(HBox([month_year_dropdown, problem_dropdown, min_policy_count_input, max_policy_count_input]))
display(VBox([plot_output_policies, plot_output_payouts, plot_output_net_revenue])) # Display all three plots

# Trigger initial plot display
update_plot()

In [ ]:
import pandas as pd

# --- 1. Data Type Conversion (Crucial for numerical comparison) ---
# Convert 'INS_PAYOUT_IN_Rs' and 'NET_DEALER_PAYOUT' columns to numeric types.
# This handles cases where values might be strings due to formatting (e.g., commas, currency symbols).
# .astype(str) ensures all values are treated as strings before replacement.
# .str.replace('[^0-9.-]', '', regex=True) removes any character that is NOT a digit, a period (.), or a hyphen (-).
# errors='coerce' will turn any values that cannot be parsed into a number (after cleaning) into NaN.
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
df['NET_DEALER_PAYOUT'] = pd.to_numeric(df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

# --- NEW: Identify Master Dealers with only one unique Branch ---
if 'BRANCH NAME' not in df.columns:
    raise ValueError("Error: 'BRANCH NAME' column not found in your DataFrame. This column is required for the requested filtering.")

# Count unique branches per Master Dealer
dealer_branch_counts = df.groupby('Master_Dealer_Name')['BRANCH NAME'].nunique().reset_index()
dealer_branch_counts.rename(columns={'BRANCH NAME': 'Unique_Branch_Count'}, inplace=True)

# Filter for dealers that worked with only one branch
single_branch_dealers = dealer_branch_counts[dealer_branch_counts['Unique_Branch_Count'] == 1]['Master_Dealer_Name']

# --- End NEW section ---

# --- 2. Calculate Total Unique Policies Issued by each Master Dealer ---
# Group the DataFrame by 'Master_Dealer_Name' and count the number of unique 'Insurance_Policy_No' for each dealer.
total_policies_issued = df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
# Rename the column to match the desired output format.
total_policies_issued.rename(columns={'Insurance_Policy_No': 'NO OF POLICY ISSUED IN TOTAL'}, inplace=True)

# --- 3. Filter Data based on the Condition (INS_PAYOUT_IN_Rs < NET_DEALER_PAYOUT) ---
# Create a filtered DataFrame containing only the rows where the specified condition is met.
# Also, ensure that both payout values are not NaN (missing) to avoid errors during comparison.
condition_satisfied_df = df[
    (df['INS_PAYOUT_IN_Rs'] < df['NET_DEALER_PAYOUT']) &
    (df['INS_PAYOUT_IN_Rs'].notna()) &
    (df['NET_DEALER_PAYOUT'].notna())
]

# --- 4. Count Unique Policies that Satisfied the Condition for each Master Dealer ---
# Group the *filtered* DataFrame by 'Master_Dealer_Name' and count unique 'Insurance_Policy_No' that met the condition.
policies_satisfying_condition = condition_satisfied_df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index()
# Rename the column to match the desired output format.
policies_satisfying_condition.rename(columns={'Insurance_Policy_No': 'CONDITION SATISFIED'}, inplace=True)

# --- 5. Merge the two counts and determine "PROBLEM" status ---
# Perform a left merge to combine the 'total_policies_issued' with 'policies_satisfying_condition'.
# A left merge ensures all master dealers from the total list are included, even if they have no policies satisfying the condition.
final_report = pd.merge(
    total_policies_issued,
    policies_satisfying_condition,
    on='Master_Dealer_Name',
    how='left'
)

# Fill any NaN values in 'CONDITION SATISFIED' (which occur if a dealer had no policies meeting the condition) with 0.
# Then convert the column to an integer type for cleaner display.
final_report['CONDITION SATISFIED'] = final_report['CONDITION SATISFIED'].fillna(0).astype(int)

# Create the 'PROBLEM' column based on the comparison of the two counts.
# If the total policies issued matches the policies satisfying the condition, it's 'SOLVED', otherwise 'FAILED'.
final_report['PROBLEM'] = final_report.apply(
    lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
    axis=1
)

# --- NEW: Filter final_report to include only Master Dealers with a single branch ---
final_report_single_branch = final_report[final_report['Master_Dealer_Name'].isin(single_branch_dealers)]
# --- End NEW section ---

# --- 6. Display the final report in a proper table format ---
print("--- Master Dealer Policy Analysis (Single Branch Only) ---")
print(final_report_single_branch.to_string(index=False)) # .to_string(index=False) for clean, non-indexed output

# --- 7. Feature to Export the Report ---
output_file_name = "Master_Dealer_Policy_Analysis_Report_Single_Branch.xlsx"
try:
    final_report_single_branch.to_excel(output_file_name, index=False)
    print(f"\nReport successfully exported to '{output_file_name}'")
    print(f"You can find this file in the same directory where your Python script is running.")
except Exception as e:
    print(f"\nError exporting report to Excel: {e}")
    print("Please ensure the file is not open and you have write permissions to the directory.")

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import IntText, Dropdown, Output, VBox, HBox
from IPython.display import display, clear_output

# --- Perform initial data processing (outside update_plot for efficiency) ---
# 1. Data Type Conversion for Payouts
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
df['NET_DEALER_PAYOUT'] = pd.to_numeric(df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

# Add NET_REVENUE conversion here. If NET_REVENUE is a calculated field,
# perform the calculation here as well. For example:
# df['NET_REVENUE'] = df['NET_DEALER_PAYOUT'] - df['SOME_COST_COLUMN']
if 'NET_REVENUE' in df.columns:
    df['NET_REVENUE'] = pd.to_numeric(df['NET_REVENUE'].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')
else:
    print("Warning: 'NET_REVENUE' column not found. Creating dummy data for demonstration.")
    df['NET_REVENUE'] = df['NET_DEALER_PAYOUT'] * 0.8 # Example dummy calculation

# --- Convert 'Month' (DD-MM-YYYY) column to 'Month_Year' format ---
date_column_name_in_df = 'Month'

if date_column_name_in_df in df.columns:
    df[date_column_name_in_df] = pd.to_datetime(df[date_column_name_in_df], format='%d-%m-%Y', errors='coerce')
    df['Month_Year'] = df[date_column_name_in_df].dt.strftime('%b %Y')
    df.dropna(subset=[date_column_name_in_df], inplace=True)
else:
    raise ValueError(f"Error: Date column '{date_column_name_in_df}' not found in your DataFrame. Please ensure this column exists and contains dates in 'DD-MM-YYYY' format.")

# Ensure 'BRANCH NAME' column exists for single-branch filtering
if 'BRANCH NAME' not in df.columns:
    raise ValueError("Error: 'BRANCH NAME' column not found in your DataFrame. This column is required for the requested filtering by single branch.")


# ----------------------------------------------------------------------------------

# --- Create ipywidgets for filters ---
unique_month_periods = df[date_column_name_in_df].dt.to_period('M').unique().tolist()
unique_month_periods.sort()
month_year_options = [str(m.strftime('%b %Y')) for m in unique_month_periods]

month_year_dropdown = Dropdown(
    options=['All'] + month_year_options,
    value='All',
    description='Month & Year:'
)

problem_dropdown = Dropdown(
    options=['All', 'SOLVED', 'FAILED'],
    value='All',
    description='Problem Status:'
)

min_policy_count_input = IntText(
    value=0,
    description='Min Policies:',
    continuous_update=False
)

max_policy_count_input = IntText(
    value=int(df['Insurance_Policy_No'].nunique()),
    description='Max Policies:',
    continuous_update=False
)

# New dropdown for single branch filter
single_branch_filter_dropdown = Dropdown(
    options=[('All Dealers', False), ('Only Single Branch Dealers', True)],
    value=False,
    description='Branch Filter:'
)


plot_output_policies = Output() # Output for the first plot (Policy Counts)
plot_output_payouts = Output()  # Output for the second plot (Payout Sums)
plot_output_net_revenue = Output() # Output for the third plot (Net Revenue)

# --- Function to update all plots based on filter selections ---
def update_plot(*args):
    # Clear all output areas
    with plot_output_policies:
        clear_output(wait=True)
    with plot_output_payouts:
        clear_output(wait=True)
    with plot_output_net_revenue:
        clear_output(wait=True)

    # Get filter values
    selected_month_year = month_year_dropdown.value
    selected_problem = problem_dropdown.value
    min_count = min_policy_count_input.value
    max_count = max_policy_count_input.value
    filter_by_single_branch = single_branch_filter_dropdown.value # Get new filter value

    # Start with a copy of the original DataFrame for filtering
    filtered_df_base = df.copy()

    # Apply Month & Year filter
    if selected_month_year != 'All':
        filtered_df_base = filtered_df_base[filtered_df_base['Month_Year'] == selected_month_year]

    # --- Apply Single Branch Filter (NEW) ---
    if filter_by_single_branch:
        # Calculate unique branches per Master Dealer within the *currently filtered data*
        dealer_branch_counts_current_view = filtered_df_base.groupby('Master_Dealer_Name')['BRANCH NAME'].nunique().reset_index()
        single_branch_dealers_current_view = dealer_branch_counts_current_view[
            dealer_branch_counts_current_view['BRANCH NAME'] == 1
        ]['Master_Dealer_Name']

        # Filter the base DataFrame by these single-branch dealers
        filtered_df_base = filtered_df_base[filtered_df_base['Master_Dealer_Name'].isin(single_branch_dealers_current_view)]
    # --- End Single Branch Filter ---

    if filtered_df_base.empty:
        with plot_output_policies:
            print(f"No data for selected combination of filters. Please adjust filters.")
        with plot_output_payouts:
            print(f"No data for selected combination of filters. Please adjust filters.")
        with plot_output_net_revenue:
            print(f"No data for selected combination of filters. Please adjust filters.")
        return

    # --- Data Processing for Policy Counts (First Plot) ---
    # Group by both Master_Dealer_Name and BRANCH NAME to preserve branch info for hover
    temp_total_policies_issued = filtered_df_base.groupby(['Master_Dealer_Name', 'BRANCH NAME'])['Insurance_Policy_No'].nunique().reset_index()
    temp_total_policies_issued.rename(columns={'Insurance_Policy_No': 'NO OF POLICY ISSUED IN TOTAL'}, inplace=True)

    temp_condition_satisfied_df = filtered_df_base[
        (filtered_df_base['INS_PAYOUT_IN_Rs'] < filtered_df_base['NET_DEALER_PAYOUT']) &
        (filtered_df_base['INS_PAYOUT_IN_Rs'].notna()) &
        (filtered_df_base['NET_DEALER_PAYOUT'].notna())
    ]
    temp_policies_satisfying_condition = temp_condition_satisfied_df.groupby(['Master_Dealer_Name', 'BRANCH NAME'])['Insurance_Policy_No'].nunique().reset_index()
    temp_policies_satisfying_condition.rename(columns={'Insurance_Policy_No': 'CONDITION SATISFIED'}, inplace=True)

    temp_final_report = pd.merge(
        temp_total_policies_issued,
        temp_policies_satisfying_condition,
        on=['Master_Dealer_Name', 'BRANCH NAME'],
        how='left'
    )

    temp_final_report['CONDITION SATISFIED'] = pd.to_numeric(temp_final_report['CONDITION SATISFIED'], errors='coerce')
    temp_final_report['CONDITION SATISFIED'] = temp_final_report['CONDITION SATISFIED'].fillna(0).astype(int)

    temp_final_report['PROBLEM'] = temp_final_report.apply(
        lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
        axis=1
    )

    # Apply Problem Status filter to temp_final_report
    if selected_problem != 'All':
        temp_final_report = temp_final_report[temp_final_report['PROBLEM'] == selected_problem]

    # Apply Policy Count filter to temp_final_report
    temp_final_report = temp_final_report[
        (temp_final_report['NO OF POLICY ISSUED IN TOTAL'] >= min_count) &
        (temp_final_report['NO OF POLICY ISSUED IN TOTAL'] <= max_count)
    ]

    # --- Data Processing for Payout Sums (Second Plot) ---
    # Ensure payout sums are based on the filtered_df_base (which now includes single branch filter)
    payouts_summed = filtered_df_base.groupby('Master_Dealer_Name')[
        ['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT']
    ].sum().reset_index()

    payouts_for_plot = pd.merge(
        temp_final_report[['Master_Dealer_Name', 'PROBLEM']].drop_duplicates(),
        payouts_summed,
        on='Master_Dealer_Name',
        how='inner'
    )

    # --- Data Processing for NET_REVENUE Sum (Third Plot) ---
    # Ensure net revenue sums are based on the filtered_df_base (which now includes single branch filter)
    net_revenue_summed = filtered_df_base.groupby('Master_Dealer_Name')[
        'NET_REVENUE'
    ].sum().reset_index()

    net_revenue_for_plot = pd.merge(
        temp_final_report[['Master_Dealer_Name', 'PROBLEM']].drop_duplicates(),
        net_revenue_summed,
        on='Master_Dealer_Name',
        how='inner'
    )

    # Check if data exists after all filtering before plotting
    if temp_final_report.empty:
        with plot_output_policies:
            print("No data to display for the selected combination of filters for Policy Counts.")
        with plot_output_payouts:
            print("No data to display for the selected combination of filters for Payout Sums.")
        with plot_output_net_revenue:
            print("No data to display for the selected combination of filters for Net Revenue Sum.")
        return

    # Melt the filtered data for policy counts plot
    melted_df_policies = temp_final_report.melt(
        id_vars=['Master_Dealer_Name', 'PROBLEM', 'BRANCH NAME'],
        value_vars=['NO OF POLICY ISSUED IN TOTAL', 'CONDITION SATISFIED'],
        var_name='Metric',
        value_name='Count'
    )
    melted_df_policies = melted_df_policies.sort_values(by=['Master_Dealer_Name', 'Metric'])


    # Melt the filtered data for payout sums plot
    if payouts_for_plot.empty:
        with plot_output_payouts:
            print("No data to display for the selected combination of filters for Payout Sums.")
    melted_df_payouts = payouts_for_plot.melt(
        id_vars=['Master_Dealer_Name', 'PROBLEM'],
        value_vars=['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT'],
        var_name='Metric',
        value_name='Sum'
    )
    melted_df_payouts = melted_df_payouts.sort_values(by=['Master_Dealer_Name', 'Metric'])

    # Melt the filtered data for net revenue plot
    if net_revenue_for_plot.empty:
        with plot_output_net_revenue:
            print("No data to display for the selected combination of filters for Net Revenue Sum.")
    melted_df_net_revenue = net_revenue_for_plot.melt(
        id_vars=['Master_Dealer_Name', 'PROBLEM'],
        value_vars=['NET_REVENUE'],
        var_name='Metric',
        value_name='Sum'
    )
    melted_df_net_revenue = melted_df_net_revenue.sort_values(by=['Master_Dealer_Name'])


    # --- Construct dynamic title suffix for all plots ---
    title_suffix_parts = [
        f"Month/Year={selected_month_year}",
        f"Problem={selected_problem}",
        f"Policies={min_count}-{max_count}"
    ]
    if filter_by_single_branch:
        title_suffix_parts.append("Single Branch Only")
    title_suffix = ", ".join(title_suffix_parts)


    # --- Plotting the first chart (Policy Counts) ---
    with plot_output_policies:
        fig_policies = px.bar(
            melted_df_policies,
            x='Master_Dealer_Name',
            y='Count',
            color='Metric',
            barmode='group',
            title=f'Policy Comparison: Total vs. Condition-Satisfied<br><sup>{title_suffix}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Count': 'Number of Unique Policies',
                'Metric': 'Policy Type'
            },
            hover_data={
                'Master_Dealer_Name': True,
                'Metric': True,
                'Count': True,
                'PROBLEM': True,
                'BRANCH NAME': True # Added 'BRANCH NAME' to hover_data
            },
            category_orders={"Metric": ['NO OF POLICY ISSUED IN TOTAL', 'CONDITION SATISFIED']},
            color_discrete_map={
                'NO OF POLICY ISSUED IN TOTAL': 'steelblue',
                'CONDITION SATISFIED': 'darkorange'
            }
        )
        fig_policies.update_layout(
            xaxis_title_text='Master Dealer Name', yaxis_title_text='Number of Unique Policies',
            font=dict(size=10),
            height=1000, # Consistent height
            width=1500,  # Consistent width
            title_x=0.5, hovermode="x unified", legend_title_text="Policy Type"
        )
        fig_policies.update_xaxes(tickangle=-45)
        fig_policies.update_traces(texttemplate='%{y}', textposition='outside')
        fig_policies.show()

    # --- Plotting the second chart (Payout Sums) ---
    with plot_output_payouts:
        fig_payouts = px.bar(
            melted_df_payouts,
            x='Master_Dealer_Name',
            y='Sum',
            color='Metric',
            barmode='group',
            title=f'Payout Comparison: INS_PAYOUT_IN_Rs vs. NET_DEALER_PAYOUT<br><sup>{title_suffix}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Sum': 'Total Amount (INR)',
                'Metric': 'Payout Type'
            },
            hover_data={
                'Master_Dealer_Name': True, 'Metric': True, 'Sum': True, 'PROBLEM': True
            },
            category_orders={
                "Metric": ['INS_PAYOUT_IN_Rs', 'NET_DEALER_PAYOUT']
            },
            color_discrete_map={
                'INS_PAYOUT_IN_Rs': 'mediumseagreen',
                'NET_DEALER_PAYOUT': 'darkred'
            }
        )
        fig_payouts.update_layout(
            xaxis_title_text='Master Dealer Name', yaxis_title_text='Total Amount (INR)',
            font=dict(size=10),
            height=1000, # Consistent height
            width=1500,  # Consistent width
            title_x=0.5, hovermode="x unified", legend_title_text="Payout Type"
        )
        fig_payouts.update_xaxes(tickangle=-45)
        fig_payouts.update_traces(texttemplate='%{y:.2s}', textposition='outside')
        fig_payouts.show()

    # --- Plotting the third chart (NET_REVENUE Sum) ---
    with plot_output_net_revenue:
        fig_net_revenue = px.bar(
            melted_df_net_revenue,
            x='Master_Dealer_Name',
            y='Sum',
            color_discrete_sequence=['purple'],
            title=f'Total NET_REVENUE by Master Dealer<br><sup>{title_suffix}</sup>',
            labels={
                'Master_Dealer_Name': 'Master Dealer',
                'Sum': 'Total NET_REVENUE (INR)',
                'Metric': 'Revenue Type'
            },
            hover_data={
                'Master_Dealer_Name': True, 'Sum': True, 'PROBLEM': True
            }
        )
        fig_net_revenue.update_layout(
            xaxis_title_text='Master Dealer Name', yaxis_title_text='Total NET_REVENUE (INR)',
            font=dict(size=10),
            height=1000, # Consistent height
            width=1500,  # Consistent width
            title_x=0.5, hovermode="x unified"
        )
        fig_net_revenue.update_xaxes(tickangle=-45)
        fig_net_revenue.update_traces(texttemplate='%{y:.2s}', textposition='outside')
        fig_net_revenue.show()


# --- Attach update triggers to widgets ---
month_year_dropdown.observe(update_plot, names='value')
problem_dropdown.observe(update_plot, names='value')
min_policy_count_input.observe(update_plot, names='value')
max_policy_count_input.observe(update_plot, names='value')
single_branch_filter_dropdown.observe(update_plot, names='value') # Observe new filter

# --- Initial display of widgets and plots ---
display(HBox([month_year_dropdown, problem_dropdown, min_policy_count_input, max_policy_count_input, single_branch_filter_dropdown]))
display(VBox([plot_output_policies, plot_output_payouts, plot_output_net_revenue])) # Display all three outputs

# Trigger initial plot display
update_plot()

In [ ]:
import pandas as pd

# --- Data Type Conversion (Crucial for numerical comparison) ---
# Optimize: Chained operations for direct conversion
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(
    df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True),
    errors='coerce'
)
df['NET_DEALER_PAYOUT'] = pd.to_numeric(
    df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True),
    errors='coerce'
)

# --- Identify Master Dealers with only one unique Branch (Optimized) ---
if 'BRANCH NAME' not in df.columns:
    raise ValueError("Error: 'BRANCH NAME' column not found in your DataFrame. This column is required for the requested filtering.")

# Count unique branches per Master Dealer and filter for single-branch dealers
single_branch_dealers = (
    df.groupby('Master_Dealer_Name')['BRANCH NAME']
    .nunique()
    .loc[lambda x: x == 1] # Filter directly for count == 1
    .index.tolist()       # Get the Master_Dealer_Names as a list
)
print(f"Identified {len(single_branch_dealers)} Master Dealers who have worked with only a single branch across all time.")


# --- Calculate Total Unique Policies Issued by each Master Dealer (Optimized) ---
total_policies_issued = df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index(name='NO OF POLICY ISSUED IN TOTAL')

# --- Filter Data based on the Condition (INS_PAYOUT_IN_Rs < NET_DEALER_PAYOUT) ---
condition_satisfied_df = df[
    (df['INS_PAYOUT_IN_Rs'] < df['NET_DEALER_PAYOUT']) &
    df['INS_PAYOUT_IN_Rs'].notna() &
    df['NET_DEALER_PAYOUT'].notna()
]

# --- Count Unique Policies that Satisfied the Condition for each Master Dealer (Optimized) ---
policies_satisfying_condition = (
    condition_satisfied_df.groupby('Master_Dealer_Name')['Insurance_Policy_No']
    .nunique()
    .reset_index(name='CONDITION SATISFIED')
)

# --- Merge the two counts and determine "PROBLEM" status ---
final_report = pd.merge(
    total_policies_issued,
    policies_satisfying_condition,
    on='Master_Dealer_Name',
    how='left'
).assign(
    # Fill NaN and convert to int directly in assign
    **{'CONDITION SATISFIED': lambda x: x['CONDITION SATISFIED'].fillna(0).astype(int)}
).assign(
    # Create the 'PROBLEM' column
    PROBLEM=lambda x: x.apply(
        lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
        axis=1
    )
)

# --- Filter final_report to include only Master Dealers with a single branch ---
final_report_single_branch = final_report[final_report['Master_Dealer_Name'].isin(single_branch_dealers)]


# --- Display the final report in a proper table format ---
print("\n--- Master Dealer Policy Analysis (Single Branch Only) ---")
# Use .to_string() for better console display of the full DataFrame
print(final_report_single_branch.to_string(index=False))

# --- NEW: Small Table of Solved & Failed Counts ---
print("\n--- Summary of Problem Status (Single Branch Dealers) ---")
problem_summary = final_report_single_branch['PROBLEM'].value_counts().reset_index()
problem_summary.columns = ['Status', 'Count']
print(problem_summary.to_string(index=False))


# --- Feature to Export the Report ---
output_file_name = "Master_Dealer_Policy_Analysis_Report_Single_Branch.xlsx"
try:
    with pd.ExcelWriter(output_file_name, engine='xlsxwriter') as writer:
        # Write the main report sheet
        final_report_single_branch.to_excel(writer, sheet_name='Single Branch Report', index=False)

        # Write the summary table to a separate sheet or below the main table
        problem_summary.to_excel(writer, sheet_name='Summary', index=False)

    print(f"\nReport successfully exported to '{output_file_name}'")
    print(f"You can find this file in the same directory where your Python script is running.")
except Exception as e:
    print(f"\nError exporting report to Excel: {e}")
    print("Please ensure the file is not open and you have write permissions to the directory.")

In [ ]:
import pandas as pd

# --- Data Type Conversion ---
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(
    df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True),
    errors='coerce'
)
df['NET_DEALER_PAYOUT'] = pd.to_numeric(
    df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True),
    errors='coerce'
)

# --- Identify Master Dealers with only one unique Branch ---
if 'BRANCH NAME' not in df.columns:
    raise ValueError("Error: 'BRANCH NAME' column not found in your DataFrame. This column is required for the requested filtering.")

single_branch_dealers = (
    df.groupby('Master_Dealer_Name')['BRANCH NAME']
    .nunique()
    .loc[lambda x: x == 1]
    .index.tolist()
)

# --- Calculate Total Unique Policies Issued ---
total_policies_issued = df.groupby('Master_Dealer_Name')['Insurance_Policy_No'].nunique().reset_index(name='NO OF POLICY ISSUED IN TOTAL')

# --- Filter Data based on the Condition ---
condition_satisfied_df = df[
    (df['INS_PAYOUT_IN_Rs'] < df['NET_DEALER_PAYOUT']) &
    df['INS_PAYOUT_IN_Rs'].notna() &
    df['NET_DEALER_PAYOUT'].notna()
]

# --- Count Unique Policies that Satisfied the Condition ---
policies_satisfying_condition = (
    condition_satisfied_df.groupby('Master_Dealer_Name')['Insurance_Policy_No']
    .nunique()
    .reset_index(name='CONDITION SATISFIED')
)

# --- Merge the two counts and determine "PROBLEM" status ---
final_report = pd.merge(
    total_policies_issued,
    policies_satisfying_condition,
    on='Master_Dealer_Name',
    how='left'
).assign(
    **{'CONDITION SATISFIED': lambda x: x['CONDITION SATISFIED'].fillna(0).astype(int)}
).assign(
    PROBLEM=lambda x: x.apply(
        lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
        axis=1
    )
)

# --- Filter final_report to include only Master Dealers with a single branch ---
final_report_single_branch = final_report[final_report['Master_Dealer_Name'].isin(single_branch_dealers)]

# --- GET THE LIST OF UNIQUE DEALERS WHO HAVE SATISFIED THE CONDITION AS 'SOLVED' ---
solved_dealers = final_report_single_branch[final_report_single_branch['PROBLEM'] == 'SOLVED']['Master_Dealer_Name'].unique().tolist()

print("\n--- List of Unique Master Dealers with 'SOLVED' status (from Single Branch Dealers) ---")
if solved_dealers:
    for dealer in solved_dealers:
        print(dealer)
    print(f"\nTotal Solved Dealers: {len(solved_dealers)}")
else:
    print("No Master Dealers found with 'SOLVED' status under the 'Single Branch Only' filter.")

# You can also use this list for further analysis or display
# print(solved_dealers) # Uncomment this line to see the list directly

In [ ]:
import pandas as pd

# --- Data Type Conversion ---
df['INS_PAYOUT_IN_Rs'] = pd.to_numeric(
    df['INS_PAYOUT_IN_Rs'].astype(str).str.replace('[^0-9.-]', '', regex=True),
    errors='coerce'
)
df['NET_DEALER_PAYOUT'] = pd.to_numeric(
    df['NET_DEALER_PAYOUT'].astype(str).str.replace('[^0-9.-]', '', regex=True),
    errors='coerce'
)

# --- Identify Master Dealers with only one unique Branch (across all time) ---
if 'BRANCH NAME' not in df.columns:
    raise ValueError("Error: 'BRANCH NAME' column not found in your DataFrame. This column is required for the requested filtering.")

# Count unique branches per Master Dealer and filter for truly single-branch dealers
truly_single_branch_dealers = (
    df.groupby('Master_Dealer_Name')['BRANCH NAME']
    .nunique()
    .loc[lambda x: x == 1]
    .index.tolist()
)
print(f"Identified {len(truly_single_branch_dealers)} Master Dealers who have worked with only a single branch across all time.")


# --- Calculate Total Unique Policies Issued by each Master Dealer AND Branch ---
# Group by both Master_Dealer_Name and BRANCH NAME to retain branch information
total_policies_issued_with_branch = (
    df.groupby(['Master_Dealer_Name', 'BRANCH NAME'])['Insurance_Policy_No']
    .nunique()
    .reset_index(name='NO OF POLICY ISSUED IN TOTAL')
)

# --- Filter Data based on the Condition (INS_PAYOUT_IN_Rs < NET_DEALER_PAYOUT) ---
condition_satisfied_df = df[
    (df['INS_PAYOUT_IN_Rs'] < df['NET_DEALER_PAYOUT']) &
    df['INS_PAYOUT_IN_Rs'].notna() &
    df['NET_DEALER_PAYOUT'].notna()
]

# --- Count Unique Policies that Satisfied the Condition for each Master Dealer AND Branch ---
policies_satisfying_condition_with_branch = (
    condition_satisfied_df.groupby(['Master_Dealer_Name', 'BRANCH NAME'])['Insurance_Policy_No']
    .nunique()
    .reset_index(name='CONDITION SATISFIED')
)

# --- Merge the two counts and determine "PROBLEM" status (with Branch Info) ---
# Merge on both Master_Dealer_Name and BRANCH NAME
full_dealer_branch_report = pd.merge(
    total_policies_issued_with_branch,
    policies_satisfying_condition_with_branch,
    on=['Master_Dealer_Name', 'BRANCH NAME'],
    how='left'
).assign(
    **{'CONDITION SATISFIED': lambda x: x['CONDITION SATISFIED'].fillna(0).astype(int)}
).assign(
    PROBLEM=lambda x: x.apply(
        lambda row: 'SOLVED' if row['NO OF POLICY ISSUED IN TOTAL'] == row['CONDITION SATISFIED'] else 'FAILED',
        axis=1
    )
)

# --- Filter the full report to include ONLY truly single-branch dealers ---
# This ensures we only consider dealers identified as single-branch across all data
filtered_single_branch_dealers_report = full_dealer_branch_report[
    full_dealer_branch_report['Master_Dealer_Name'].isin(truly_single_branch_dealers)
]

# --- Now, filter this report for 'SOLVED' problems ---
solved_single_branch_dealers_details = filtered_single_branch_dealers_report[
    filtered_single_branch_dealers_report['PROBLEM'] == 'SOLVED'
]

# --- Display the requested table ---
print("\n--- Details of Master Dealers with 'SOLVED' status (Single Branch Only) ---")
if not solved_single_branch_dealers_details.empty:
    print(solved_single_branch_dealers_details.to_string(index=False))
else:
    print("No Master Dealers found with 'SOLVED' status under the 'Single Branch Only' filter.")

# --- Summary of Problem Status (Single Branch Dealers) ---
print("\n--- Summary of Problem Status (Single Branch Dealers) ---")
# This summary will still be based on the overall single branch filtered report (including FAILED ones)
problem_summary = filtered_single_branch_dealers_report['PROBLEM'].value_counts().reset_index()
problem_summary.columns = ['Status', 'Count']
print(problem_summary.to_string(index=False))
